## 39. 混合ガウス分布（Gaussian Mixture Model）

### <font color = blue>**1.** </font> 簡易な例を自己実装

In [ ]:
# 参考：https://qiita.com/ta-ka/items/3e8b127620ac92a32864

import numpy as np
import matplotlib.pyplot as plt
import sys

In [ ]:
# 正規分布にノイズを乗せたデータを生成

def create_data(N, K, seed):
  np.random.seed(seed=seed)
  X, mu_star, sigma_star = [], [], []
  for i in range(K):
    loc = (np.random.rand() - 0.5) * 10.0 # range: -5.0 - 5.0
    scale = np.random.rand() * 3.0 # range: 0.0 - 3.0
    X = np.append(X, np.random.normal(loc = loc, scale = scale, size = int(N / K)))
    mu_star = np.append(mu_star, loc)
    sigma_star = np.append(sigma_star, scale)
  return (X, mu_star, sigma_star)

In [ ]:
# ノイズなしの正規分布

def gaussian(mu, sigma):
  def f(x):
    return np.exp(-0.5 * (x - mu) ** 2 / sigma) / np.sqrt(2 * np.pi * sigma)
  return f

In [ ]:
# 事後確率の尤度

def estimate_posterior_likelihood(X, pi, gf):
  l = np.zeros((X.size, pi.size))
  for (i, x) in enumerate(X):
    l[i, :] = gf(x)
  return pi * l * np.vectorize(lambda y: 1 / y)(l.sum(axis = 1).reshape(-1, 1))

In [ ]:
# ガウス分布を仮定した時のパラメータを推定

def estimate_gmm_parameter(X, gamma):
  N = gamma.sum(axis = 0)
  mu = (gamma * X.reshape((-1, 1))).sum(axis = 0) / N
  sigma = (gamma * (X.reshape(-1, 1) - mu) ** 2).sum(axis = 0) / N
  pi = N / X.size
  return (mu, sigma, pi)

In [ ]:
# Q関数を計算

def calc_Q(X, mu, sigma, pi, gamma):
  Q = (gamma * (np.log(pi * (2 * np.pi * sigma) ** (-0.5)))).sum()
  for (i, x) in enumerate(X):
    Q += (gamma[i, :] * (-0.5 * (x - mu) ** 2 / sigma)).sum()
  return Q

In [ ]:
# data
K = 3 ###
N = 10000 * K
seed = 9  ###
X, mu_star, sigma_star = create_data(N, K, seed)

plt.figure(figsize=(12,6))
n, bins, _ = plt.hist(X, 200, density=True, alpha = 0.3)
plt.show()

In [ ]:
# termination condition
epsilon = 1e-10 ###

In [ ]:
# initialize gmm parameter
pi = np.random.rand(K)
mu = np.random.randn(K)
sigma = np.abs(np.random.randn(K))
Q = -sys.float_info.max
delta = None

In [ ]:
# EM algorithm

i = 1
while delta == None or delta >= epsilon:
  gf = gaussian(mu, sigma)

  # E step: estimate posterior probability of hidden variable gamma
  gamma = estimate_posterior_likelihood(X, pi, gf)

  # M step: miximize Q function by estimating mu, sigma and pi
  mu, sigma, pi = estimate_gmm_parameter(X, gamma)

  # calculate Q function
  Q_new = calc_Q(X, mu, sigma, pi, gamma)
  delta = Q_new - Q
  print("{}th calc. delta : ".format(i), delta)
  Q = Q_new
  i +=1

In [ ]:
# result
print(u'mu*: %s, simga*: %s' % (str(np.sort(np.around(mu_star, 3))), str(np.sort(np.around(sigma_star, 3)))))
print(u'mu : %s, sgima : %s' % (str(np.sort(np.around(mu, 3))), str(np.sort(np.around(sigma, 3)))))

In [ ]:
# plot

plt.figure(figsize=(12,6))
n, bins, _ = plt.hist(X, 200, density=True, alpha = 0.3)
seq = np.arange(-10, 10, 0.01)
for i in range(K):
  plt.plot(seq, gaussian(mu[i], sigma[i])(seq)/K, linewidth = 2.0)

plt.show()

### <font color = blue>**2.** </font> GMMによるクラスタリング

In [ ]:
# 参考：https://qiita.com/isuya/items/018a0867bdc95033736d

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter

In [ ]:
# irisのデータを使用する

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
df = sns.load_dataset('iris')
ax1.scatter(df['petal_length'], df['petal_width'], color='gray')
ax1.set_title('without label')
ax1.set_xlabel('petal_length')
ax1.set_ylabel('petal_width')

for sp in df['species'].unique():
  x = df[df['species'] == sp]['petal_length']
  y = df[df['species'] == sp]['petal_width']
  ax2.scatter(x, y, label=sp)

ax2.legend()
ax2.set_title('with label')
ax2.set_xlabel('petal_length')
ax2.set_ylabel('petal_width')
plt.show()

In [ ]:
# クラスタのデータ数をカウントできる配列の実装

class ClusterArray(object):
  def __init__(self, array):
    # arrayは1次元のリスト、配列
    self._array = np.array(array, dtype=np.int)
    self._counter = Counter(array)

  @property
  def array(self):
    return self._array.copy()

  def count(self, k):
    return self._counter[k]

  def __setitem__(self, i, k):
    # 実行されるとself._counterも更新される
    pre_value = self._array[i]
    if pre_value == k:
      return

    if self._counter[pre_value] > 0:
      self._counter[pre_value] -= 1
    self._array[i] = k
    self._counter[k] += 1

  def __getitem__(self, i):
    return self._array[i]

In [ ]:
# 確率密度関数を計算する際の計算量を減らした

def log_deformed_gaussian(x, mu, var):
  norm_squared = ((x - mu) * (x - mu)).sum(axis=1)
  return -norm_squared / (2 * var)

In [ ]:
## scikit-learn風に自己実装

class GaussianMixtureClustering(object):
  def __init__(self, K, D, var=1, var_pri=1, seed=None):
    self.K = K  # クラスタ数
    self.D = D  # 説明変数の次元(実装しやすたのため、コンストラクタの時点で設定しておく)
    self.z = None

    # 確率分布のパラメータ設定
    self.mu = np.zeros((self.K, self.D))
    self.var = var  # 固定、すべてのクラスタで共通
    self.pi = np.full(self.K, 1 / self.K)  # 固定、すべてのクラスタで共通

    # 事前分布の設定
    self.mu_pri = np.zeros(self.D)
    self.var_pri = var_pri

    self._random = np.random.RandomState(seed)

  def fit(self, X, n_iter):
    init_z = self._random.randint(0, self.K, X.shape[0])
    self.z = ClusterArray(init_z)

    for _ in range(n_iter):
      for k in range(self.K):
        self.mu[k] = self._sample_mu_k(X, k)
      for i, x_i in enumerate(X):
        self.z[i] = self._sample_zi(x_i)

  def _sample_zi(self, x_i):
    log_probs_xi = log_deformed_gaussian(x_i, self.mu, self.var)

    probs_zi = np.exp(log_probs_xi) * self.pi
    probs_zi = probs_zi / probs_zi.sum()

    z_i = self._random.multinomial(1, probs_zi)
    z_i = np.where(z_i)[0][0]
    return z_i

  def _sample_mu_k(self, X, k):
    xk_bar = np.array([x for i, x in enumerate(X) if self.z[i] == k]).mean(axis=0)
    var_pos = 1 / (self.z.count(k) / self.var + 1 / self.var_pri)
    mu_pos = var_pos * (xk_bar * self.z.count(k) / self.var + self.mu_pri / self.var_pri)

    mu_k = self._random.multivariate_normal(mu_pos, var_pos * np.eye(self.D))
    return mu_k

In [ ]:
# データセットの読み込み
df = sns.load_dataset('iris')
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values

In [ ]:
# 混合ガウスモデルによるクラスタリング
gmc = GaussianMixtureClustering(K=3, D=4, var=0.1, seed=1)
gmc.fit(X, n_iter=10)

In [ ]:
# 結果を図示する際の色分けラベル調整

raw = gmc.z.array
copy = raw.copy()
copy[raw==0] = 2
copy[raw==1] = 0
copy[raw==2] = 1

df['GMM_cluster'] = copy

In [ ]:
# 結果の可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
for sp in df['species'].unique():
  x = df[df['species'] == sp]['petal_length']
  y = df[df['species'] == sp]['petal_width']
  ax1.scatter(x, y, label=sp)

ax1.legend()
ax1.set_title('species')
ax1.set_xlabel('petal_length')
ax1.set_ylabel('petal_width')

for k in range(gmc.K):
  x = df[df['GMM_cluster'] == k]['petal_length']
  y = df[df['GMM_cluster'] == k]['petal_width']
  ax2.scatter(x, y, label=k)

ax2.legend()
ax2.set_title('GMM cluster')
ax2.set_xlabel('petal_length')
ax2.set_ylabel('petal_width')
plt.show()

### <font color = blue>**3.** </font> ライブラリ使用例 </font>

$\downarrow \downarrow$ 公式リファレンス $\downarrow \downarrow$\
https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.datasets import load_iris

In [ ]:
iris_data = load_iris()
X = iris_data.data
Y = iris_data.target
plt.scatter(X[:,2], X[:,3], c=Y)
plt.show()

In [ ]:
gm = GaussianMixture(3)
gm.fit(X)
Y_pred = gm.predict(X)

copy = Y_pred.copy()
copy[raw==0] = 2
copy[raw==1] = 0
copy[raw==2] = 1

plt.scatter(X[:,2], X[:,3], c=copy)
plt.show()

## 40. 因子分析（Factor Analysis）

### <font color = blue>**1.** </font> EMアルゴリズムによる因子分析のパラメータ推定

In [ ]:
# 出典: https://qiita.com/m1t0/items/a79bc97559b66b3f8adc

import numpy as np
import pandas as pd
import scipy.stats as sp

In [ ]:
def gaussian(xvec, mean, cov):
  return 1/((2*np.pi)**(xvec.size/2)*(np.sqrt(np.linalg.det(cov)))) * np.exp((-1/2)*np.dot((xvec - mean).T, 
                                                                                           np.dot(np.linalg.inv(cov), 
                                                                                                  (xvec - mean))))

In [ ]:
def likelihood(X, mean, cov):
  sum = .0
  for i in range(len(X)):
    sum += gaussian(X[i], mean, cov)
  return np.log(sum)

In [ ]:
def EMforFA(data, m, starts=10):  # m: number of factors
  X = data.astype("float64")
  n, p = X.shape
  #data標準化
  X = sp.stats.zscore(X)

  likeAll = -1000
  Lhat = np.zeros((p, m))
  Psihat = np.zeros((p, p))

  for j in range(starts):
    turn = 0
    print("---------- epoc: %d -------" % (j+1))
    # Lambda, Psi の初期化
    L = np.random.uniform(0.1, 0.3, p*m).reshape(p, m)
    for k in range(p):
      L[k, np.random.randint(0, m-1)] = np.random.uniform(0.6, 0.9, 1)
    Psi = np.diag(np.diag(np.identity(p) - np.dot(L, L.T)))
    like = likelihood(X, np.zeros(p), Psi)

    while True:
      # print(turn, ": ", like)
      ## E-step
      beta = np.dot(L.T, np.linalg.inv(Psi + np.dot(L, L.T)))
      Ezx = np.zeros((m, n))
      for a in range(n):
        Ezx[:, a] = np.dot(beta, X[a, :].T)
      Ez2x = np.zeros((n, m, m))
      for a in range(n):
        Ez2x[a][:, :] = np.identity(m) - np.dot(beta, L) + np.outer(np.dot(beta, X[a, :].T), np.dot(X[a, :], beta.T))
      sumL  = np.zeros((p, m))
      sumL2 = np.zeros((m, m))
      sumX = np.zeros((p, p))
      sumZX = np.zeros((p, p))
      for i in range(n):
        sumL += np.outer(X[i, :].T, Ezx[:, i].T)
        sumL2 += Ez2x[i][:, :]
        sumX += np.outer(X[i, :].T, X[i, :])
      #M-step 
      L = np.dot(sumL, np.linalg.inv(sumL2))
      for i in range(n):
        sumZX += np.dot(L, np.outer(Ezx[:, i], X[i, :]))
      Psi = np.diag(np.diag(sumX - sumZX)) / n

      like_new = likelihood(X, np.zeros(p), Psi)
      diff = like_new - like
      if diff < 10**(-6) and diff >= 0:
        if likeAll < like:
          Lhat = L
          Psihat = Psi
          break
      if diff < 0:
        print("diff is negative")
        break
      like = like_new
      if np.isnan(like):
        print("the calculated likelihood is nan")
        break
      turn += 1
  return (Lhat, Psihat)

In [ ]:
## データを準備
# 出処 : http://bstat.jp/material/
# 5因子性格テストデータ 190(参加者)×25(項目)の評定値

X = pd.read_excel("https://github.com/jiai-edu/teaching_ep_elementary-phython/blob/master/datasets/Big5.xls?raw=true", 
                  sheet_name="Big5", 
                  usecols=np.arange(1, 26))
X = X.values  # .as_matrix() -> .values

In [ ]:
A, D = EMforFA(X, 5)

In [ ]:
A.shape

In [ ]:
print(A)

In [ ]:
D.shape

In [ ]:
print(D)

### <font color = blue>**2.** </font> ライブラリ使用例

$\downarrow \downarrow$ 公式リファレンス $\downarrow \downarrow$\
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FactorAnalysis.html

In [ ]:
# 出典: https://qiita.com/y_itoh/items/227cb33317ceb09199c2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# 12教科のテストの得点が1000人分、というダミーデータを因子分析用に作成されている

url = 'https://raw.githubusercontent.com/yumi-ito/sample_data/master/subject_scores.csv'
df = pd.read_csv(url)
df.head()

In [ ]:
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

#標本数(count)、平均値(mean)、標準偏差(std)、最小値(min)、四分位数(25%, 50%, 75%)、最大値(max)

In [ ]:
# matplotlibを日本語表示に対応させるモジュール
!pip install japanize-matplotlib
import japanize_matplotlib

In [ ]:
# 散布図行列を表示
pd.plotting.scatter_matrix(df, figsize=(18, 18), alpha=0.8)
plt.show()

In [ ]:
# sklearnの標準化モジュールをインポート
from sklearn.preprocessing import StandardScaler

In [ ]:
# データを変換する計算式を生成
sc = StandardScaler()
sc.fit(df)

# 実際にデータを変換
z = sc.transform(df)

In [ ]:
z.shape

In [ ]:
print(z)

In [ ]:
# sklearnのFactorAnalysis(因子分析)クラスをインポート
from sklearn.decomposition import FactorAnalysis as FA

In [ ]:
# 因子数を指定
n_components = 3  ###

# 因子分析の実行
fa = FA(n_components, max_iter=5000) # モデルを定義
fitted = fa.fit_transform(z) # fitとtransformを一括処理

In [ ]:
fitted.shape

In [ ]:
print(fitted)

In [ ]:
# 因子負荷量行列を取得
fa.components_.T

In [ ]:
# 変数Factor_loading_matrixに格納
Factor_loading_matrix = fa.components_.T

# データフレームに変換
pd.DataFrame(Factor_loading_matrix, 
             columns=["第1因子", "第2因子", "第3因子"], 
             index=[df.columns])

- 第１因子は、すべての変数がプラスになっているので、この因子が高くなると全変数が足並みをそろえて高くなる
  - 「総合学力」とか「基礎学力」などと命名できそう
- 第２因子は、国語・英語の絶対値が大きく、いずれもプラスで、また倫理もプラスでやや高めとなっている
  - 「言語能力」と命名しておきます
- 第３因子は、数学・物理の絶対値がずば抜けて大きく、いずれもプラスになっている
  - この因子は「数理能力」といえます

## 41. 多様体学習（Manifold learning）

### <font color = blue>**1.** </font> 自己組織化マップ（SOM : Self-Organizing Map）

In [ ]:
# 出典: https://shizenkarasuzon.hatenablog.com/entry/2019/02/03/174551

SOMとは
- 自己組織化マップ（SOP、Self-organizing maps） はフィンランドの研究者、T. Kohonenが発明したニューラルネットの一種です
- SOMは教師なし学習を行い、高次元のデータセットを低次元空間（主に二次元）に写像するのに使用されます
- 2次元に写像した場合、データ分布が地図（Topographic Map）のように可視化されるため、この地図をデータマイニングに用いられることがよくあります

紹介するサンプル
- SOMは多次元のデータを可視化するのに適しています
  - 「色」という三次元データを分類・可視化してみたいと思います
- パソコンの世界では、「色」はRed、Blue、Greenの三色の混ぜ方で決まっています
  - それぞれ0～255の強さがあります
  - 例えば、（Red, Green, Blue)= (255,0,0)は赤色、(Red, Green, Blue) = (255,255,0)は赤と緑の光の混ぜ合わせなので黄色となります
  - つまり、色には三次元のデータあるのです

- この三次元のデータをSOMに組み込み、似た色のデータを近くに配置することで、色を分類することができるのではないか？というのがSOMのアルゴリズムです

アルゴリズム
- 最初はノードにランダムな色を記憶させておきます
  - これは、隣同士のノードの関係がよくわからない、つまり学習が行われていない初期段階です
- まず、SOMには、大量の色データが一つずつ順番に与えられます
- それら一つ一つのデータに対して学習していくのですが、その学習アルゴリズムは以下の2STEPです
  1. 入力された色データに最も近い色を表しているノードを探す
  2. そのノードと、その周辺ノードを、入力された色データの色に近づける
    - 例えば、入力データが赤色だった場合、赤色のノードとその周りがさらに赤っぽくなります
- この2STEPを繰り返すことで、だんだんとノードがきれいになっていきます
- そして、最終的には、色が二次元マップ上にきれいに分類されます

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def som(ColorVec):
  min_index = np.argmin(((weight-ColorVec)**2).sum(axis=2))
  mini = int(min_index / Ny)
  minj = int(min_index % Ny)
  for i in range(-2,3):
    for j in range(-2,3):
      try:
        weight[mini+i,minj+j] += alpha * (ColorVec - weight[mini+i,minj+j])
      except:
        pass

In [ ]:
Nx = 30
Ny = 30
learntime = 10000
alpha = 0.08

In [ ]:
weight = np.random.random([Nx,Ny,3])

for time in range(learntime):
  ColorVec = np.random.rand(3)
  som(ColorVec)
  if time%100 == 0 and time<1001: ####
    print("transition@{}".format(time))
    im0 = plt.imshow(weight, interpolation='none')
    plt.show()

print("result")
im2 = plt.imshow(weight, interpolation='none')
plt.show()

### <font color = blue>**2.** </font> ライブラリ使用例

$\downarrow \downarrow$ 公式リファレンス $\downarrow \downarrow$\
https://scikit-learn.org/stable/modules/manifold.html



In [ ]:
!pip install ugtm

In [ ]:
!pip install somoclu

In [ ]:
!pip install umap-learn

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.manifold import SpectralEmbedding
from sklearn.manifold import Isomap
from sklearn.manifold import TSNE
from scipy.sparse.csgraph import connected_components

from ugtm import eGTM
from somoclu import Somoclu
from umap import UMAP

In [ ]:
dataset = datasets.load_digits()
X = dataset.data
Y = dataset.target

In [ ]:
methods = {
    'PCA': PCA(),
    'MDS': MDS(),
    'Isomap': Isomap(),
    'LLE': LocallyLinearEmbedding(),
    'Laplacian Eigenmaps': SpectralEmbedding(),
    't-SNE': TSNE(),
    'UMAP': UMAP(),
    'GTM': eGTM(),
    }

In [ ]:
fig = plt.figure(figsize=(18, 18))

for i, (name, method) in enumerate(methods.items()):
  t0 = time()
  Z = method.fit_transform(X)
  t1 = time()
  ax = fig.add_subplot(3, 3, i + 1)
  ax.scatter(Z[:, 0], Z[:, 1],c=Y,cmap='Paired', vmin=0, vmax=12)
  ax.set_title("%s (%.2g sec)" % (name, t1 - t0))

t0 = time()
som = Somoclu(64,64)
som.train(data=X, epochs=100)
map = som.get_surface_state()
Z = som.get_bmus(map)

t1 = time()
ax = fig.add_subplot(3, 3, 9)
ax.scatter(x=Z[:,0],y=Z[:,1],c=Y,cmap='Paired', vmin=0, vmax=12)
ax.set_title("%s (%.2g sec)" % ('SOM', t1 - t0))
plt.show()

In [ ]:
from collections import OrderedDict
from functools import partial
from time import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
from sklearn import manifold, datasets

In [ ]:
# Next line to silence pyflakes. This import is needed.
Axes3D
n_points = 1000
X, color = datasets.make_s_curve(n_points, random_state=0)
n_neighbors = 10
n_components = 2

In [ ]:
methods = {
    'PCA': PCA(),
    'MDS': MDS(),
    'Isomap': Isomap(),
    'LLE': LocallyLinearEmbedding(),
    'Laplacian Eigenmaps': SpectralEmbedding(),
    't-SNE': TSNE(),
    'UMAP': UMAP(),
    'GTM': eGTM(),
    }

In [ ]:
# Create figure
fig = plt.figure(figsize=(20, 9))

# Add 3d scatter plot
ax = fig.add_subplot(251, projection='3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.Spectral)
ax.view_init(4, -72)

# Plot results
for i, (label, method) in enumerate(methods.items()):
  t0 = time()
  Z = method.fit_transform(X)
  t1 = time()
  ax = fig.add_subplot(2, 5, 2 + i )
  ax.scatter(Z[:, 0], Z[:, 1], c=color, cmap=plt.cm.Spectral)
  ax.set_title("%s (%.2g sec)" % (label, t1 - t0))
  ax.xaxis.set_major_formatter(NullFormatter())
  ax.yaxis.set_major_formatter(NullFormatter())
  ax.axis('tight')

t0 = time()
som = Somoclu(32,32)
som.train(data=X, epochs=100)
map = som.get_surface_state()
Z = som.get_bmus(map)
t1 = time()

ax = fig.add_subplot(2, 5, 10)
plt.scatter(Z[:,0],Z[:,1], c=color, cmap=plt.cm.Spectral)
ax.set_title("%s (%.2g sec)" % ('SOM', t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.axis('tight')
plt.show()